## Setup

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import qutip as qt
from qutip.qip.operations import *
from qutip.qip.circuit import *
import sys
sys.path.append('../SimulationCode/')
from BlindGatesSimulation import *
from fiber_network import FiberNetwork
from SiVnodes import SiV
from SiVgates import *
from Plots import *
import pandas as pd
import glob


In [3]:
# colors for plots
c1 = '#F15F57'
c2 = '#F6851F'
c3 = '#FDB913'
c4 = '#743062'
c5 = '#C87EB5'
c6 = '#4CC0B3'
c7 = '#1C9AAA'

#### Define SiVs 

In [4]:
# Create SiVs:

#Server A, G12
siv_a = SiV(kappa_in= (74.9 - 54.5)*(10**3), kappa_w= (54.5)*(10**3), g=5.6*(10**3), wCav = (0)*(10**3), 
             wSiv = -(479.8 -639.6)*(10**3), dwEl = 0.5*(10**3)) # G12
#Server B, B16
siv_b = SiV(kappa_in= (43.5 - 26.0)*(10**3), kappa_w= (26.0)*(10**3), g=8.5*(10**3), wCav = (0)*(10**3), 
             wSiv = -(804.9 -657.6)*(10**3), dwEl = -0.5*(10**3)) # B16

### Setup single Node experiments in B16

In [5]:
# Create Networks:
b16_network = FiberNetwork(siv_b) # this device is now in B16 fridge

# Create Simulation:
sim = BlindComputing(b16_network)

In [6]:
# Setup efficiencies
fudge = 1

b16_network.fibercoupling_eff = 0.6*fudge
b16_network.tdi_eff = 0.35*0.3
b16_network.snspd_eff = 0.9
b16_network.detection_eff
b16_network.detection_eff_reset()

print('Detection efficientvy is = ', b16_network.detection_eff)

Detection efficientvy is =  0.0567


#### MW gates

In [7]:
# gate fidelities

b16_mwfid = 0.99
g12_mwfid = 1

#### Move the SiV to the desired contrast point

In [8]:
b16_contrast = 25 # range 18 - 30
g12_contrast = 20 # range 12 - 20

siv_b.set_contrast(b16_contrast)
actual_contrast_b16 = siv_b.get_best_contrast()
print("B16 contrast is set to = ", actual_contrast_b16)

new contrast 27.979731852612378
B16 contrast is set to =  27.979731852612378


## B16 single qubit rotations

In [108]:
imperfections ={'contrast_noise': 0, #(0 not noisy or 1 noisy)
                'contrast': 'real', #'real'
                'tdinoise': - 0.1, #np.pi/25, #'real'
                'mw': 'real', # or perfect
                'mw_noise': 1, #(0 is stable or 1 noisy/underotates overroates every experimental shot)
                'mw_fid_num': [b16_mwfid, b16_mwfid] # default fidelities
               }
mu = 0.05

In [109]:
phi1 = [0, np.pi/4, np.pi/2, 3*np.pi/4]
phi2 = 0
phi3 = 0
el_initial_xp  = qt.ket2dm((qt.basis(2,0)+ qt.basis(2,1)).unit())

In [ ]:
cluster_state_length = 1
n_rounds = 10000

rates_apd1_apd2 = np.empty((0, 2), dtype=float)
rho_phi_n_array_s = np.empty((0, n_rounds, 2, 2), dtype=complex)
rho_phi_n_array_cl = np.empty((0, n_rounds, 2, 2), dtype=complex)
nxnynz_cl_phi_array =  np.empty((0, n_rounds, 3), dtype=float)
nxnynz_s_phi_array =  np.empty((0, n_rounds, 3), dtype=float)
for j in range(len(phi1)):
    rho_n_array_cl = np.empty((0, 2, 2), dtype=complex)
    rho_n_array_s = np.empty((0, 2, 2), dtype=complex)
    rho_click = np.empty((0, 1), dtype=int)
    phi1e = phi1[j]
    print("Phi angle = ", phi1e)
    nxnynz_cl_array =  np.empty((0, 3), dtype=float)
    nxnynz_s_array =  np.empty((0, 3), dtype=float)
    for i in range(n_rounds): 
        if i%100 == 0:
            print(i)
        rho_init_xp = sim.single_node_electron_exp(el_initial_xp, imperfections, cluster_state_length, phi1e, phi1, phi3, mu)
        rho_n_item_s = rho_init_xp[0]
        rho_n_item_cl = (qt.sigmaz()**(rho_init_xp[1]))*rho_init_xp[0]*(qt.sigmaz()**(rho_init_xp[1]))
        rho_n_array_s = np.vstack([rho_n_array_s, [rho_n_item_s]])
        rho_n_array_cl = np.vstack([rho_n_array_cl, [rho_n_item_cl]])
        nxnynz_cl = calculate_bloch_components(rho_n_item_cl)
        nxnynz_s = calculate_bloch_components(rho_n_item_s)
        nxnynz_cl_array =  np.vstack([nxnynz_cl_array, [nxnynz_cl]])
        nxnynz_s_array =  np.vstack([nxnynz_s_array, [nxnynz_s]])
    nxnynz_cl_phi_array =  np.vstack([nxnynz_cl_phi_array, [nxnynz_cl_array]])
    nxnynz_s_phi_array =  np.vstack([nxnynz_s_phi_array, [nxnynz_s_array]])
    mean_cl_nxnynz = np.mean(nxnynz_cl_array, axis =0)
    std_cl_nxnynz = np.std(nxnynz_cl_array, axis =0)/np.sqrt(n_rounds)
    mean_s_nxnynz = np.mean(nxnynz_s_array, axis =0)
    std_s_nxnynz = np.std(nxnynz_s_array, axis =0)/np.sqrt(n_rounds)
    print(mean_cl_nxnynz, std_cl_nxnynz)
    print(mean_s_nxnynz, std_s_nxnynz)
    rates_apd1_apd2 = np.append(rates_apd1_apd2, [rho_init_xp[2:4]])

    rho_phi_n_array_s = np.vstack([rho_phi_n_array_s, [rho_n_array_s]])
    rho_phi_n_array_cl = np.vstack([rho_phi_n_array_cl, [rho_n_array_cl]])

In [118]:
fid_err_phi =  np.empty((0, 2), dtype=float)
fid_ave_phi =  np.empty((0, 2), dtype=float)
for j in range(len(phi1)):
    U = rz(phi1[j])
    el_rho_final_ideal = U*el_initial_xp*U.dag()
    fid_n =  np.empty((0, 2), dtype=float)
    for k in range(n_rounds):
        fid = (qt.fidelity(qt.Qobj(rho_phi_n_array_cl[j][k]), el_rho_final_ideal))**2
        fid_n = np.append(fid_n, fid)
    fid_ave = np.mean(fid_n, axis = 0)
    # this is only the variability of the measurement
    fid_ave_std = np.std(fid_n, axis = 0)/(np.sqrt(n_rounds))
    fid_ave_phi = np.append(fid_ave_phi, fid_ave)
    fid_err_phi = np.append(fid_err_phi, fid_ave_std)

In [136]:
print(np.mean(nxnynz_cl_phi_array, axis = 1))
print(np.std(nxnynz_cl_phi_array, axis = 1)/np.sqrt(n_rounds))
print(np.mean(nxnynz_s_phi_array, axis = 1))
print(np.std(nxnynz_s_phi_array, axis = 1)/np.sqrt(n_rounds))

[[ 0.91115257 -0.09011496 -0.0069548 ]
 [ 0.71044618  0.56911975 -0.01335258]
 [ 0.09455779  0.89694037 -0.00762512]
 [-0.58053426  0.69644991 -0.00639887]]
[[0.00095105 0.0003485  0.00325048]
 [0.00148431 0.00166445 0.00276787]
 [0.00318744 0.00040197 0.0019758 ]
 [0.00204214 0.00163996 0.00254271]]
[[ 0.3057609   0.00047353 -0.0069548 ]
 [ 0.31610002  0.00475654 -0.01335258]
 [ 0.30978769  0.0088627  -0.00762512]
 [ 0.315204   -0.00804258 -0.00639887]]
[[0.00863571 0.00096618 0.00325048]
 [0.00653335 0.00592941 0.00276787]
 [0.00120707 0.00897797 0.0019758 ]
 [0.00528554 0.00715453 0.00254271]]


In [120]:
fid = np.mean(fid_ave_phi, axis = 0)
fiderr = np.sqrt(np.sum(fid_err_phi**2)/len(fid_err_phi))
print('Fidelity over phi = ', fid, "+-",  fiderr)

Fidelity over phi =  0.9519808652203994 +- 0.00034877246933622664


In [121]:
rho_ave_phi_array =  np.empty((0, 2, 2), dtype=float)
eigen_phi_std_array = np.empty((0, 2), dtype=float)
eig_eta_array = np.empty((0, 2), dtype=float)

mean_nxnynz = np.mean(nxnynz_s_phi_array, axis =1)
std_nxnynz = np.std(nxnynz_s_phi_array, axis =1)/np.sqrt(n_rounds)

# eigen ebnergies and uncertainties of rho of phis
eigen_phis = [(1/2)*(1 -np.sqrt(np.sum(mean_nxnynz**2, axis = 1))), (1/2)*(1 +np.sqrt(np.sum(mean_nxnynz**2, axis = 1)))]
eigen_phis_std = np.array([eigenvalue_uncertainty(mean_nxnynz[i], std_nxnynz[i]) for i in range(len(mean_nxnynz))])

rho_ave_phi_array = np.mean(rho_phi_n_array_s, axis = 1)
rho_ave_tot = np.mean(rho_ave_phi_array, axis = 0)
nxnynz_tot = np.array([calculate_bloch_components(qt.Qobj(rho_ave_tot))])
nxnynz_tot_std = np.sqrt(np.sum(eigen_phis_std**2))/len(eigen_phis_std)

# std of variability of eigenvalues of rho tot
std_eigen_tot = eigenvalue_uncertainty(nxnynz_tot.ravel(), [nxnynz_tot_std,nxnynz_tot_std,nxnynz_tot_std])
eigen_tot = np.abs(qt.Qobj(rho_ave_tot).eigenenergies())

hv_tot = qt.entropy_vn(qt.Qobj(rho_ave_tot)) - np.mean([qt.entropy_vn(qt.Qobj(rho)) for rho in rho_ave_phi_array])
rho_tot_sigma_lambdas = (std_eigen_tot, std_eigen_tot)
rho_tot_lambdas = (eigen_tot[0], eigen_tot[1])
rho_lambdas = list(zip(eigen_phis[0], eigen_phis[1]))
rho_sigma_lambdas =  [(value, value) for value in eigen_phis_std]

hv_std = holevo_bound_uncertainty(rho_tot_lambdas, rho_tot_sigma_lambdas, rho_lambdas, rho_sigma_lambdas)

print('Holevo bound is = ', hv_tot, hv_std)


Holevo bound is =  3.4100408453419107e-05 0.000983889713131907


## B16 universal single qubit rotations

In [11]:
imperfections ={'contrast_noise': 0, #(0 not noisy or 1 noisy)
                'contrast': 'real', #'real'
                'tdinoise': - 0.1, #np.pi/25, #'real'
                'mw': 'real', # or perfect
                'mw_noise': 1, #(0 is stable or 1 noisy/underotates overroates every experimental shot)
                'mw_fid_num': [b16_mwfid, b16_mwfid] # default fidelities
               }
mu = 0.05
phi1 = [0]
phi2 = [0]
phi3 = [0]
el_initial_xp  = qt.ket2dm((qt.basis(2,0)+ qt.basis(2,1)).unit())
cluster_state_length = 2
n_rounds = 1


In [ ]:
def single_qubit_universal_blind_gate(el_initial, imperfections, cluster_state_length, phi_array, mu):
    # define hadamart
    Had = ry(-np.pi/2) # Ry for hadamard

    measurement_outputs = np.empty((0, 2), dtype=float)

    rho = el_initial
    for i in range(cluster_state_length): 
        rho_after_SPG = single_rotation_electron(self, rho, imperfections, phi_array[i], mu)
        rho = Had*rho_after_SPG[0]*Had.dag()
        measurement_outputs = np.append(measurement_outputs, [rho_after_SPG[2::]])

    return rho, measurement_outputs

In [ ]:
# final correction: Z gate
el_rho_semifinal1 = (sigmaz()**(m1))*el_rho_3*(sigmaz()**(m1))
el_rho_semifinal2 = (sigmax()**m2)*el_rho_semifinal1*(sigmax()**m2)
el_rho_final = (sigmaz()**(m3))*el_rho_semifinal2*(sigmaz()**(m3))

## New structure: Single gate fidelities and blindness as a function of MW errors

In [ ]:
mw_list = np.linspace(1, 0.7, 12)
phi1 = [0, np.pi/4, np.pi/2, 3*np.pi/4]
phi2 = 0
phi3 = 0
el_initial_xp  = qt.ket2dm((qt.basis(2,0)+ qt.basis(2,1)).unit())
# el_initial_xp  = qt.ket2dm((qt.basis(2,0)+ 1j*qt.basis(2,1)).unit())

In [ ]:
# redoing the structure to make calculations faster. Scan mw first, then phi then n rounds 
# do calculations on all of the matricies, don't save
# do errors properly

imperfections ={'contrast_noise': 0, #(0 not noisy or 1 noisy)
                'contrast': 'perfect', #'real'
                'mw': 'real', # or perfect
                'mw_noise': 1, #(0 is stable or 1 noisy/underotates overroates every experimental shot)
                'mw_fid_num': [1, 1] # default fidelities
               }

# move SiV in B16 to a new contrast location
# contrast = 50
# siv_b.set_contrast(contrast)
# actual_contrast = siv_b.get_best_contrast()

cluster_state_length = 1

n_rounds = 500
n_col = 6
mu = 0.00001

rates_apd1_apd2_angle = np.empty((0, 2), dtype=float)
rho_mw_phi_n_array_s = np.empty((0, len(phi1), n_rounds, 2, 2), dtype=complex)
rho_mw_phi_n_array_cl = np.empty((0, len(phi1), n_rounds, 2, 2), dtype=complex)

#phi list
for i in range(len(mw_list)) :
    imperfections['mw_fid_num']= [mw_list[i], mw_list[i]]
    print(mw_list[i])
    # my list
    
    rates_apd1_apd2 = np.empty((0, 2), dtype=float)
    rho_phi_n_array_s = np.empty((0, n_rounds, 2, 2), dtype=complex)
    rho_phi_n_array_cl = np.empty((0, n_rounds, 2, 2), dtype=complex)

    for j in range(len(phi1)):
        rho_n_array_cl = np.empty((0, 2, 2), dtype=complex)
        rho_n_array_s = np.empty((0, 2, 2), dtype=complex)
        rho_click = np.empty((0, 1), dtype=int)
        phi1e = phi1[j]
        print("Phi angle = ", phi1e)
        for i in range(n_rounds): 
            rho_init_xp, Xxp_c, Yxp_c, Zxp_c, Xxp_s, Yxp_s, Zxp_s = sim.single_node_electron_exp(el_initial_xp, imperfections, cluster_state_length, phi1e, phi1, phi3, mu)
            rho_n_array_s = np.append(rho_n_array_s, [rho_init_xp[0]], axis = 0)
            rho_n_array_cl = np.append(rho_n_array_cl, [(qt.sigmaz()**(rho_init_xp[1]))*rho_init_xp[0]*(qt.sigmaz()**(rho_init_xp[1]))], axis = 0)
        rates_apd1_apd2 = np.append(rates_apd1_apd2, [rho_init_xp[2:4]])
        
        rho_phi_n_array_s = np.vstack([rho_phi_n_array_s, [rho_n_array_s]])
        rho_phi_n_array_cl = np.vstack([rho_phi_n_array_cl, [rho_n_array_cl]])
    
    rho_mw_phi_n_array_s= np.vstack([rho_mw_phi_n_array_s, [rho_phi_n_array_s]])
    rho_mw_phi_n_array_cl= np.vstack([rho_mw_phi_n_array_cl, [rho_phi_n_array_cl]])



In [ ]:
# rho_mw_phi_n_array_s_buffer = rho_mw_phi_n_array_s
# rho_mw_phi_n_array_cl_buffer = rho_mw_phi_n_array_cl

#### Fidelity of the gates

In [ ]:
## compare to ideal results

fid_err =  np.empty((0, 2), dtype=float)
fid_ave =  np.empty((0, 2), dtype=float)

for i in range(len(mw_list)):
    fid_err_phi_mw =  np.empty((0, 2), dtype=float)
    fid_ave_phi_mw =  np.empty((0, 2), dtype=float)
    for j in range(len(phi1)):
        U = rz(phi1[j])
        el_rho_final_ideal = U*el_initial_xp*U.dag()
        fid_mw =  np.empty((0, 2), dtype=float)
        for k in range(n_rounds):
            fid = (qt.fidelity(qt.Qobj(rho_mw_phi_n_array_cl[i][j][k]), el_rho_final_ideal))**2
            fid_mw = np.append(fid_mw, fid)
        fid_ave_mw = np.mean(fid_mw, axis = 0)
        # this is only the variability of the measurement
        fid_ave_std = np.std(fid_mw, axis = 0)/(np.sqrt(n_rounds))
        fid_ave_phi_mw = np.append(fid_ave_phi_mw, fid_ave_mw)
        fid_err_phi_mw = np.append(fid_err_phi_mw, fid_ave_std)
    
    fid_ave = np.append(fid_ave, fid_ave_phi_mw)
    fid_err = np.append(fid_err, fid_err_phi_mw)
    

In [ ]:

plt.figure(figsize=(8, 4))
plt.plot(mw_list, fid_ave[::len(phi1)], color=c1)
plt.errorbar(mw_list,  fid_ave[::len(phi1)], yerr= fid_err[::len(phi1)],  fmt='o', color=c1, label='phi=0')
plt.plot(mw_list, fid_ave[1::len(phi1)], color=c2)
plt.errorbar(mw_list,  fid_ave[1::len(phi1)], yerr= fid_err[1::len(phi1)],  fmt='o', color=c2, label='phi=pi/4')
plt.plot(mw_list, fid_ave[2::len(phi1)], color=c3)
plt.errorbar(mw_list,  fid_ave[2::len(phi1)], yerr= fid_err[2::len(phi1)],  fmt='o', color=c3, label='phi=pi/2')
plt.plot(mw_list, fid_ave[3::len(phi1)], color=c4)
plt.errorbar(mw_list,  fid_ave[3::len(phi1)], yerr= fid_err[3::len(phi1)],  fmt='o', color=c4, label='phi=3pi/4')

#Add labels and title
plt.xlabel('Mw gate fidelity')
plt.ylabel('fidelity')
plt.title('fidelity of single qubit rotation init xp')
plt.legend()
plt.savefig('/Users/azizasuleymanzade/Lukin SiV Dropbox/Aziza azizasuleymanzade@g.harvard.edu/bqc_paper/Simulation/SimulationCode/OutputFiles/BlindComputing/Fig1/Figures/PaperFigs/Fid_xpinit_mu0002_vs_mw.pdf', format='pdf')   # Save as PDF


# Show the plot
plt.show()

#### Checking mw fidelities when noise is incorporates

In [ ]:
fidel_values_pi_pi2 = {'pi': 0.86,
                'pi_half': 0.86
                }

pi_fid_arr_n =  np.empty((0, 2), dtype=float)
pi2_fid_arr_n =  np.empty((0, 2), dtype=float)
n_rounds_array = [5, 10, 50 , 100, 200, 500]

for n_rounds in n_rounds_array:
    pi_fid_arr =  np.empty((0, 2), dtype=float)
    pi2_fid_arr =  np.empty((0, 2), dtype=float)
    for i in range(n_rounds):
        print(i)
        gates = set_mw_fidelities('real', 1, fidel_values_pi_pi2)
        pi_fid = qt.fidelity(gates['pi']*qt.basis(2,0), qt.basis(2,1))**2
        pi2_fid = qt.fidelity(gates['pi_half']*qt.basis(2,0), (qt.basis(2,0)+ qt.basis(2,1)).unit())**2
        pi_fid_arr = np.append(pi_fid_arr, pi_fid)
        pi2_fid_arr = np.append(pi2_fid_arr, pi2_fid)
    pi_fid_arr_n = np.append(pi_fid_arr_n, np.array([np.mean(pi_fid_arr), np.std(pi_fid_arr)/np.sqrt(n_rounds)]))
    pi2_fid_arr_n = np.append(pi2_fid_arr_n, np.array([np.mean(pi2_fid_arr), np.std(pi2_fid_arr)/np.sqrt(n_rounds)]))




In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(n_rounds_array, pi_fid_arr_n[::2], color=c1)
plt.errorbar(n_rounds_array,  pi_fid_arr_n[::2], yerr= pi_fid_arr_n[1::2],  fmt='o', color=c1, label='phi=0')

plt.plot(n_rounds_array, pi2_fid_arr_n[::2], color=c4)
plt.errorbar(n_rounds_array,  pi2_fid_arr_n[::2], yerr= pi2_fid_arr_n[1::2],  fmt='o', color=c4, label='phi=0')



#Add labels and title
plt.xlabel('n_rounds')
plt.ylabel('fidelity')
plt.title('fidelity of single qubit rotation init xp')
plt.legend()

# Show the plot
plt.show()

In [ ]:
fid_err

In [ ]:
fid_ave

In [ ]:
## compare to ideal results

fid_err =  np.empty((0, 2), dtype=float)
fid_ave =  np.empty((0, 2), dtype=float)

for i in range(len(mw_list)):
    fid_err_phi_mw =  np.empty((0, 2), dtype=float)
    fid_ave_phi_mw =  np.empty((0, 2), dtype=float)
    for j in range(len(phi1)):
        U = rz(phi1[j])
        el_rho_final_ideal = U*el_initial_xp*U.dag()
        fid_mw =  np.empty((0, 2), dtype=float)
        for k in range(n_rounds):
            fid = (qt.fidelity(qt.Qobj(rho_mw_phi_n_array_cl[i][j][k]), el_rho_final_ideal))**2
            fid_mw = np.append(fid_mw, fid)
        fid_ave_mw = np.mean(fid_mw, axis = 0)
        # this is only the variability of the measurement
        fid_ave_std = np.std(fid_mw, axis = 0)/(np.sqrt(n_rounds))
        fid_ave_phi_mw = np.append(fid_ave_phi_mw, fid_ave_mw)
        fid_err_phi_mw = np.append(fid_err_phi_mw, fid_ave_std)
    
    fid_ave = np.append(fid_ave, fid_ave_phi_mw)
    fid_err = np.append(fid_err, fid_err_phi_mw)
    

In [ ]:
hol_err =  np.empty((0, 2), dtype=float)
hol_ave =  np.empty((0, 2), dtype=float)

for i in range(len(mw_list)):
    print(i)
    rho_ave_phi_array =  np.empty((0, 2, 2), dtype=float)
    eigen_phi_std_array = np.empty((0, 2), dtype=float)
    eig_eta_array = np.empty((0, 2), dtype=float)
    for j in range(len(phi1)):
        for k in range(n_rounds):
            nx, ny, nz = np.round(calculate_bloch_components(qt.Qobj(rho_mw_phi_n_array_s[i][j][k])), 5)
            nxnynz_array =  np.vstack([nxnynz_array, [nx, ny, nz]])
        mean_nxnynz = np.mean(nxnynz_array, axis =0)
        std_nxnynz = np.std(nxnynz_array, axis =0)/np.sqrt(n_rounds)
        std_eigen_eta = eigenvalue_uncertainty(mean_nxnynz, std_nxnynz)
        eigen_eta = [(1/2)*(1 -np.sqrt(np.sum(mean_nxnynz**2))), (1/2)*(1 +np.sqrt(np.sum(mean_nxnynz**2)))]
        print(eigen_eta, std_eigen_eta)

        eig_eta_array = np.append(eig_eta_array, np.array([eigen_eta]))
        # density state for a phi
        rho_ave_phi = np.mean(rho_mw_phi_n_array_s[i][j], axis = 0)
        rho_ave_phi_array = np.append(rho_ave_phi_array, [rho_ave_phi], axis = 0)
        eigen_phi_std_array = np.append(eigen_phi_std_array, std_eigen_eta)

    rho_ave_tot = np.mean(rho_ave_phi_array, axis = 0)
    nxnynz = np.array([calculate_bloch_components(qt.Qobj(rho_ave_tot))])
    nxnynz_std = np.sqrt(np.sum(eigen_phi_std_array**2))/len(eigen_phi_std_array)

    # std of variability of eigenvalues of rho tot
    std_eigen_eta = eigenvalue_uncertainty(nxnynz.ravel(), [nxnynz_std,nxnynz_std,nxnynz_std])
    eigen_tot = np.abs(qt.Qobj(rho_ave_tot).eigenenergies())

    hv_tot = qt.entropy_vn(qt.Qobj(rho_ave_tot)) - np.mean([qt.entropy_vn(qt.Qobj(rho)) for rho in rho_ave_phi_array])
    rho_tot_sigma_lambdas = (std_eigen_eta, std_eigen_eta)
    rho_tot_lambdas = (eigen_tot[0], eigen_tot[1])
    rho_lambdas = [(eig_eta_array[i], eig_eta_array[i+1]) for i in range(0, len(eig_eta_array), 2)] 
    rho_sigma_lambdas =  [(value, value) for value in eigen_phi_std_array]
    hv_std = holevo_bound_uncertainty(rho_tot_lambdas, rho_tot_sigma_lambdas, rho_lambdas, rho_sigma_lambdas)
    hol_ave = np.append(hol_ave, hv_tot)
    hol_err = np.append(hol_err, hv_std)


In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(mw_list, hol_ave, color=c1)
plt.errorbar(mw_list,  hol_ave, yerr= hol_err,  fmt='o', color=c1)

#Add labels and title
plt.xlabel('MW fid')
plt.ylabel('Blindness')
plt.title('Blindness of single qubit single rotation, init xp')

# Show the plot
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(mw_list, hol_ave, color=c1)
plt.errorbar(mw_list,  hol_ave, yerr= hol_err,  fmt='o', color=c1)

#Add labels and title
plt.xlabel('MW fid')
plt.ylabel('Blindness')
plt.title('Blindness of single qubit single rotation, init x+')
plt.savefig('/Users/azizasuleymanzade/Lukin SiV Dropbox/Aziza azizasuleymanzade@g.harvard.edu/bqc_paper/Simulation/SimulationCode/OutputFiles/BlindComputing/Fig1/Figures/PaperFigs/Blind_xpinit_mu0002_vs_mw.pdf', format='pdf')   # Save as PDF


# Show the plot
plt.show()

In [ ]:
hol_ave

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(mw_list, hol_ave, color=c1)
plt.errorbar(mw_list,  hol_ave, yerr= hol_err,  fmt='o', color=c1)

#Add labels and title
plt.xlabel('MW fid')
plt.ylabel('Blindness')
plt.title('Blindness of single qubit single rotation, init xp')

# Show the plot
plt.show()

In [ ]:
hol_ave

In [ ]:
mw_list

In [ ]:
hol_ave

In [ ]:
def blindness_singleRot(rho_array_cl):
        """
        Calculate the Holevo bound and its uncertainty given an array of density matrices and their standard deviations.
        
        Parameters:
        - rho_phi_array: List or array of density matrices corresponding to different phases.
        - rho_std_phi_array: List or array of standard deviations for each element in the density matrices.
        - delta: Small perturbation value for numerical derivative (default is 1e-6).
        
        Returns:
        - holevo: The calculated Holevo bound.
        - holevo_error: The uncertainty in the Holevo bound.
        """
        
        
        


        # Calculate the average density matrix
        rho_all = np.mean(rho_phi_array, axis=0)
        
        # Calculate the mean entropy of the individual density matrices
        mn = np.mean([qt.entropy_vn(qt.Qobj(rho)) for rho in rho_phi_array])

        # Calculate the Holevo bound
        holevo = qt.entropy_vn(qt.Qobj(rho_all)) - mn

        # Initialize the error in the Holevo bound
        holevo_error = 0
        
        for k, rho in enumerate(rho_phi_array):
            partial_derivatives = np.zeros(rho.shape, dtype=complex)
            chi = holevo
            
            # Calculate the partial derivatives numerically
            for i in range(rho.shape[0]):
                for j in range(rho.shape[1]):
                    perturbed_rho = rho.copy()
                    
                    # Apply perturbation and ensure Hermiticity
                    delta_rho = np.zeros(rho.shape, dtype=complex)
                    delta_rho[i, j] = delta
                    delta_rho[j, i] = np.conj(delta_rho[i, j])  # Ensure Hermiticity
                    
                    # Adjust diagonal elements to preserve trace
                    trace_adjustment = np.trace(delta_rho)
                    delta_rho[i, i] -= trace_adjustment / rho.shape[0]
                    
                    perturbed_rho += delta_rho
                    
                    # Calculate perturbed average density matrix
                    perturbed_rho_all = np.mean(
                        [perturbed_rho if idx == k else rho_phi_array[idx] for idx in range(len(rho_phi_array))],
                        axis=0
                    )
                    
                    # Recalculate mean entropy
                    perturbed_mn = np.mean([qt.entropy_vn(qt.Qobj(rho_phi)) for rho_phi in rho_phi_array])
                    
                    # Calculate perturbed Holevo bound
                    perturbed_chi = qt.entropy_vn(qt.Qobj(perturbed_rho_all)) - perturbed_mn
                    
                    # Calculate the partial derivative
                    partial_derivatives[i, j] = (perturbed_chi - chi) / delta
            
            # Sum the squared errors propagated through each partial derivative
            holevo_error += np.sum((np.abs(partial_derivatives) * rho_std_phi_array[k]) ** 2)
        
        # Take the square root to obtain the final uncertainty
        holevo_error = np.abs(np.sqrt(holevo_error))
        
        return holevo, holevo_error


In [ ]:
holevo=  np.empty((0, 2), dtype=float)
holevo_err =  np.empty((0, 2), dtype=float)

# Initialize empty arrays to accumulate results
rho_ave_mw = np.empty((0, 4, 2, 2), dtype=float)
rho_std_mw= np.empty((0, 4, 2, 2), dtype=float)

# Loop through mu_list and accumulate results 
for i in range(len(mw_list)):
    # Extract the ith element from each list and convert to an array
    rho_ave_onemw = np.array([lst[i] for lst in rho_ave_s_phis])
    rho_std_onemw = np.array([lst[i] for lst in rho_std_s_phis])

    # Append the arrays directly (as they should already have shape (2,))
    rho_ave_mw = np.append(rho_ave_mw, [rho_ave_onemw], axis=0)
    rho_std_mw = np.append(rho_std_mw, [rho_std_onemw], axis=0)

# calculate blindness as a function of mu
for j in range(len(mw_list)):
    holevo, holevo_err = blindness_singleRot(rho_ave_mw[j], rho_std_mw[j], delta=1e-6)
    holevo_mw = np.append(holevo_mw, holevo)
    holevo_err_mw = np.append(holevo_err_mw, holevo_err)


#### Save density matrices 

In [ ]:
import datetime

# Example complex matrices

# Convert the complex matrices to string representation
data = {
    'rho_ave_s': [str(rho_ave_s_phi_mw.tolist())],
    'rho_std_s': [str(rho_std_s_phi_mw.tolist())],  # Replace with actual std data
    'rho_ave_cl': [str(rho_ave_cl_phi_mw.tolist())],  # Replace with actual cl data
    'rho_std_cl': [str(rho_std_cl_phi_mw.tolist())]  # Replace with actual cl std data
}

df = pd.DataFrame(data)

# File path for the output CSV
# file_path = f'/Users/azizasuleymanzade/Lukin SiV Dropbox/Aziza azizasuleymanzade@g.harvard.edu/bqc_paper/Simulation/SimulationCode/OutputFiles/BlindComputing/Fig1/SingleGates_xpinit_rhos_phis_{n_rounds}pts_mu{mu}_mw_list_perfcontr_{datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.csv'
file_path = f'/Users/azizasuleymanzade/Lukin SiV Dropbox/Aziza azizasuleymanzade@g.harvard.edu/bqc_paper/Simulation/SimulationCode/OutputFiles/BlindComputing/Fig1/dummy.csv'

# Write the DataFrame to a CSV file
df.to_csv(file_path, index=False)


#### Retrieve the densities

In [ ]:
import ast

# File path for the input CSV
file_path = f'/Users/azizasuleymanzade/Lukin SiV Dropbox/Aziza azizasuleymanzade@g.harvard.edu/bqc_paper/Simulation/SimulationCode/OutputFiles/BlindComputing/Fig1/dummy.csv'
# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Convert the string representations back into arrays
def string_to_array(string):
    # Use ast.literal_eval to safely evaluate the string to a list
    list_repr = ast.literal_eval(string)
    # Convert the list back to a NumPy array
    return np.array(list_repr, dtype=complex)

# Apply the conversion to the relevant columns
df['rho_ave_s'] = df['rho_ave_s'].apply(string_to_array)
df['rho_std_s'] = df['rho_std_s'].apply(string_to_array)
df['rho_ave_cl'] = df['rho_ave_cl'].apply(string_to_array)
df['rho_std_cl'] = df['rho_std_cl'].apply(string_to_array)

# Now you can access the arrays
rho_ave_s = df['rho_ave_s'][0]
rho_std_s = df['rho_std_s'][0]
rho_ave_cl = df['rho_ave_cl'][0]
rho_std_cl = df['rho_std_cl'][0]



In [ ]:
# Client's data
 
rho_ave_cl_0_mw = np.array(rho_ave_cl[0:len(mw_list)])
rho_ave_cl_pi4_mw = np.array(rho_ave_cl[len(mw_list):2*len(mw_list)])
rho_ave_cl_pi2_mw = np.array(rho_ave_cl[2*len(mw_list):3*len(mw_list)])
rho_ave_cl_3pi4_mw = np.array(rho_ave_cl[3*len(mw_list):])
rho_ave_cl_phis = [rho_ave_cl_0_mw, rho_ave_cl_pi4_mw, rho_ave_cl_pi2_mw, rho_ave_cl_3pi4_mw]

rho_std_cl_0_mw = np.array(rho_std_cl[0:len(mw_list)])
rho_std_cl_pi4_mw = np.array(rho_std_cl[len(mw_list):2*len(mw_list)])
rho_std_cl_pi2_mw = np.array(rho_std_cl[2*len(mw_list):3*len(mw_list)])
rho_std_cl_3pi4_mw = np.array(rho_std_cl[3*len(mw_list):])
rho_std_cl_phis = [rho_std_cl_0_mw, rho_std_cl_pi4_mw, rho_std_cl_pi2_mw, rho_std_cl_3pi4_mw]

# Server's data
 
rho_ave_s_0_mw = np.array(rho_ave_s[0:len(mw_list)])
rho_ave_s_pi4_mw = np.array(rho_ave_s[len(mw_list):2*len(mw_list)])
rho_ave_s_pi2_mw = np.array(rho_ave_s[2*len(mw_list):3*len(mw_list)])
rho_ave_s_3pi4_mw = np.array(rho_ave_s[3*len(mw_list):])
rho_ave_s_phis = [rho_ave_s_0_mw, rho_ave_s_pi4_mw, rho_ave_s_pi2_mw, rho_ave_s_3pi4_mw]

rho_std_s_0_mw = np.array(rho_std_cl[0:len(mw_list)])
rho_std_s_pi4_mw = np.array(rho_std_cl[len(mw_list):2*len(mw_list)])
rho_std_s_pi2_mw = np.array(rho_std_cl[2*len(mw_list):3*len(mw_list)])
rho_std_s_3pi4_mw = np.array(rho_std_cl[3*len(mw_list):])
rho_std_s_phis = [rho_std_s_0_mw, rho_std_s_pi4_mw, rho_std_s_pi2_mw, rho_std_s_3pi4_mw]

In [ ]:
rho_ave_cl_phis

In [ ]:
rho_std_cl_phis

In [ ]:

plt.figure(figsize=(8, 4))
plt.plot(mw_list, fid_phi_mw[0:len(mw_list)], color=c1)
plt.errorbar(mw_list,  fid_phi_mw[0:len(mw_list)], yerr= fid_err_phi_mw[0:len(mw_list)]/np.sqrt(n_rounds),  fmt='o', color=c1, label='phi=0')
plt.plot(mw_list, fid_phi_mw[len(mw_list):2*len(mw_list)], color=c2)
plt.errorbar(mw_list,  fid_phi_mw[len(mw_list):2*len(mw_list)], yerr= fid_err_phi_mw[len(mw_list):2*len(mw_list)]/np.sqrt(n_rounds),  fmt='o', color=c2,  label='phi=pi/4')
plt.plot(mw_list, fid_phi_mw[2*len(mw_list):3*len(mw_list)], color=c3)
plt.errorbar(mw_list,  fid_phi_mw[2*len(mw_list):3*len(mw_list)], yerr= fid_err_phi_mw[2*len(mw_list):3*len(mw_list)]/np.sqrt(n_rounds),  fmt='o', color=c3, label='phi=pi/2')
plt.plot(mw_list, fid_phi_mw[3*len(mw_list):], color=c4)
plt.errorbar(mw_list,  fid_phi_mw[3*len(mw_list):], yerr= fid_err_phi_mw[3*len(mw_list):]/np.sqrt(n_rounds),  fmt='o', color=c4, label='phi=3pi/4')

#Add labels and title
plt.xlabel('Mw gate fidelity')
plt.ylabel('fidelity')
plt.title('fidelity of single qubit rotation init xp')
plt.legend()

# Show the plot
plt.show()

In [ ]:
holevo_mw

In [ ]:
holevo_err_mw

In [ ]:
rho_phi_array = rho_ave_mw[0]
# Calculate the average density matrix
rho_all = np.mean(rho_phi_array, axis=0)

# Calculate the mean entropy of the individual density matrices
mn = np.mean([qt.entropy_vn(qt.Qobj(rho)) for rho in rho_phi_array])

# Calculate the Holevo bound
holevo = qt.entropy_vn(qt.Qobj(rho_all)) - mn


In [ ]:
mn

In [ ]:
holevo

In [ ]:
mx = 0.5*qt.identity(2)
qt.entropy_vn(mx)

In [ ]:
qt.entropy_vn(qt.Qobj(rho_all))

In [ ]:
np.log(1.02)

In [ ]:
holevo_mw

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(mw_list, holevo_mw, color=c1)
plt.errorbar(mw_list,  holevo_mw, yerr= np.abs(holevo_err_mw)/np.sqrt(n_rounds),  fmt='o', color=c1)

#Add labels and title
plt.xlabel('MW fid')
plt.ylabel('Blindness')
plt.title('Blindness of single qubit single rotation, init xp')

# Show the plot
plt.show()

array([[[0.5       +0.j        , 0.17779517+0.j        ],
        [0.17779517+0.j        , 0.5       +0.j        ]],

       [[0.49474799+0.j        , 0.27918727-0.11492054j],
        [0.27918727+0.11492054j, 0.50525201+0.j        ]],

       [[0.5009652 +0.j        , 0.17103865+0.02111985j],
        [0.17103865-0.02111985j, 0.4990348 +0.j        ]],

       [[0.49946759+0.j        , 0.16007542-0.01164971j],
        [0.16007542+0.01164971j, 0.50053241+0.j        ]]])